### Getting Idea About the data

In [ ]:
# Importing Necessary Libraries
import pandas as pd
import re
from collections import Counter
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [33]:
# Reading data
df = pd.read_csv('../messy_ice_detention.csv',encoding='latin-1')
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head(10)


Rows: 140, Columns: 8


,,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,These statistics are made available to the pub...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"ICE FACILITIES DATA, FY25",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"ICE Enforcement and Removal Operations Data, F...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,This list is limited to facilities that have a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data Source: ICE Integrated Decision Support (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
6,ADAMS COUNTY DET CENTER,NATCHEZ,MS,1876.461078,266.4311377,6.724550898,4.25748503,45673
7,ADELANTO ICE PROCESSING CENTER,ADELANTO,CA,6.401197605,4.323353293,22.60479042,32.63473054,45491
8,ALAMANCE COUNTY DETENTION FACILITY,GRAHAM,NC,4.329341317,3.461077844,6.083832335,6.185628743,45554
9,ALEXA$NDRIA STAGING FACILITY,ALEXANDRIA,LA,137.5209581,47.41317365,76.00598802,52.48502994,45533


### Data Cleaning

In [34]:
# Skipping the first few rows to only necessary data
df = pd.read_csv("../messy_ice_detention.csv", skiprows=6,encoding='latin-1')
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")
df.head(10)

Rows: 134, Columns: 8


,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
0,ADAMS COUNTY DET CENTER,NATCHEZ,MS,1876.461078,266.431138,6.724551,4.257485,45673
1,ADELANTO ICE PROCESSING CENTER,ADELANTO,CA,6.401198,4.323353,22.604790,32.634731,45491
2,ALAMANCE COUNTY DETENTION FACILITY,GRAHAM,NC,4.329341,3.461078,6.083832,6.185629,45554
3,ALEXA$NDRIA STAGING FACILITY,ALEXANDRIA,LA,137.520958,47.413174,76.005988,52.485030,45533
4,ALLEGANY COUNTY JAIL,BELMONT,NY,1.221557,0.018000,0.000000,0.000000,NaN
5,ALL%EN PARISH PUBLIC SAFETY +COMPLEX,OBERLIN,LA,101.311377,30.251497,33.766467,10.664671,45638
6,ATLANTA US PEN,ATLANTA,,17.341317,3.610778,3.053892,2.724551,NaN
7,B^AKER COUNTY SHERIFF DEPT.,MACCLENNY,FL,26.221557,38.640719,88.652695,88.730539,45589
8,BERLIN FED. CORR. INST.,BERLIN,NH,1.748503,0.089800,0.970060,0.712575,NaN
9,BLUEBONNET DETENTION FACILITY,ANSON,TX,311.305389,164.550898,241.616767,104.514970,45638


#### Cleaning Name Column

In [38]:
# Replace 'N/A', 'NA', and blanks with proper NaN
df['Name'] = df['Name'].replace(
    ['N/A', 'NA', ' ', ''], pd.NA
)

df.Name.isnull().sum()

np.int64(2)

In [39]:
# Checking null rows
null_name_rows = df[df['Name'].isnull()]
null_name_rows

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
116,<NA>,ELK RIVER,MN,4.592814,5.598802,9.281437,2.760479,45414
123,<NA>,DOVER,NH,0.000000,0.000000,49.161677,41.335329,45547


In [40]:
# Now remove rows where Name is null
df = df.dropna(subset=['Name'])

In [41]:
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 132, Columns: 8


In [42]:
## Checking Unnecessary Characters in the 'Name' Column
all_names = ' '.join(df['Name'].dropna().astype(str))
special_chars = re.findall(r'[^a-zA-Z0-9\s\-]', all_names)
char_counts = Counter(special_chars)

for char, count in char_counts.items():
    print(f"'{char}': {count} times")

'$': 1 times
'%': 2 times
'+': 4 times
'^': 4 times
'.': 9 times
'(': 12 times
',': 2 times
')': 14 times
'&': 3 times
'@': 3 times
'#': 1 times
'_': 1 times
'*': 1 times
'/': 2 times


In [43]:
# Keep only letters, numbers, spaces, hyphens, periods, dot
# Add the Special Character you want to keep
df['Name'] = df['Name'].astype(str).apply(
    lambda x: re.sub(r'[^a-zA-Z0-9\s\-.]', '', x)
)

# Replace multiple spaces with a single space and trim
df['Name'] = df['Name'].str.replace(r'\s+', ' ', regex=True).str.strip()
df.head(10)

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
0,ADAMS COUNTY DET CENTER,NATCHEZ,MS,1876.461078,266.431138,6.724551,4.257485,45673
1,ADELANTO ICE PROCESSING CENTER,ADELANTO,CA,6.401198,4.323353,22.604790,32.634731,45491
2,ALAMANCE COUNTY DETENTION FACILITY,GRAHAM,NC,4.329341,3.461078,6.083832,6.185629,45554
3,ALEXANDRIA STAGING FACILITY,ALEXANDRIA,LA,137.520958,47.413174,76.005988,52.485030,45533
4,ALLEGANY COUNTY JAIL,BELMONT,NY,1.221557,0.018000,0.000000,0.000000,NaN
5,ALLEN PARISH PUBLIC SAFETY COMPLEX,OBERLIN,LA,101.311377,30.251497,33.766467,10.664671,45638
6,ATLANTA US PEN,ATLANTA,,17.341317,3.610778,3.053892,2.724551,NaN
7,BAKER COUNTY SHERIFF DEPT.,MACCLENNY,FL,26.221557,38.640719,88.652695,88.730539,45589
8,BERLIN FED. CORR. INST.,BERLIN,NH,1.748503,0.089800,0.970060,0.712575,NaN
9,BLUEBONNET DETENTION FACILITY,ANSON,TX,311.305389,164.550898,241.616767,104.514970,45638


In [44]:
df['Name'].count()

np.int64(132)

In [45]:
# Number of duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count}")

# Show duplicate rows (optional)
# df[df.duplicated()]


Duplicate rows: 0


In [46]:
empty_rows = df[df.isnull().all(axis=1)]
print(f"Completely empty rows: {len(empty_rows)}")

Completely empty rows: 0


In [47]:
null_summary = df.isnull().sum()
print("Null values per column:\n")
print(null_summary[null_summary > 0])

Null values per column:

Last Inspection End Date    17
dtype: int64


In [48]:
df.dtypes

Name                         object
City                         object
State                        object
Level A                     float64
Level B                     float64
Level C                     float64
Level D                     float64
Last Inspection End Date     object
dtype: object

#### Cleaning Date Columns

In [49]:
# Replace 'N/A', 'NA', and blanks with proper NaN
df['Last Inspection End Date'] = df['Last Inspection End Date'].replace(
    ['N/A', 'NA', ' ', ''], pd.NA
)


In [ ]:
# changing the value to the numeric value
df['Last Inspection End Date'] = pd.to_numeric(
    df['Last Inspection End Date'],
    errors='coerce'
)

In [51]:
# Convert numeric Excel date to datetime, ignoring NaNs
df['Last Inspection End Date'] = pd.to_datetime(
    df['Last Inspection End Date'],
    origin='1899-12-30',
    unit='D',
    errors='coerce'
)


In [52]:
print(df['Last Inspection End Date'].head())
print(df['Last Inspection End Date'].dtype)


0   2025-01-16
1   2024-07-18
2   2024-09-19
3   2024-08-29
4          NaT
Name: Last Inspection End Date, dtype: datetime64[ns]
datetime64[ns]


In [53]:
df.head(10)

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
0,ADAMS COUNTY DET CENTER,NATCHEZ,MS,1876.461078,266.431138,6.724551,4.257485,2025-01-16
1,ADELANTO ICE PROCESSING CENTER,ADELANTO,CA,6.401198,4.323353,22.604790,32.634731,2024-07-18
2,ALAMANCE COUNTY DETENTION FACILITY,GRAHAM,NC,4.329341,3.461078,6.083832,6.185629,2024-09-19
3,ALEXANDRIA STAGING FACILITY,ALEXANDRIA,LA,137.520958,47.413174,76.005988,52.485030,2024-08-29
4,ALLEGANY COUNTY JAIL,BELMONT,NY,1.221557,0.018000,0.000000,0.000000,NaT
5,ALLEN PARISH PUBLIC SAFETY COMPLEX,OBERLIN,LA,101.311377,30.251497,33.766467,10.664671,2024-12-12
6,ATLANTA US PEN,ATLANTA,,17.341317,3.610778,3.053892,2.724551,NaT
7,BAKER COUNTY SHERIFF DEPT.,MACCLENNY,FL,26.221557,38.640719,88.652695,88.730539,2024-10-24
8,BERLIN FED. CORR. INST.,BERLIN,NH,1.748503,0.089800,0.970060,0.712575,NaT
9,BLUEBONNET DETENTION FACILITY,ANSON,TX,311.305389,164.550898,241.616767,104.514970,2024-12-12


#### Cleaning the State Column

In [54]:
# Replace 'N/A', 'NA', and blanks with proper NaN
df['State'] = df['State'].replace(
    ['N/A', 'NA', ' ', ''], np.nan
)

In [55]:
#Checking null values
df['State'].isnull().sum()

np.int64(2)

In [56]:
# Checking null rows
null_state_rows = df[df['State'].isnull()]
null_state_rows

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
6,ATLANTA US PEN,ATLANTA,NaN,17.341317,3.610778,3.053892,2.724551,NaT
71,LA SALLE COUNTY REGIONAL DETENTION CENTER,ENCINAL,NaN,8.910180,0.706587,0.000000,0.071900,2025-02-07


##### Finding the state using the city Name

In [57]:
# Initialize geocoder
geolocator = Nominatim(user_agent="city_to_state_filler")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)



# Function to get state from city name
def get_state_from_city(city_name):
    try:
        location = geocode(city_name)
        full_address = location.address

        parts = full_address.split(',')
        state = parts[2]
        state
        
        return state
    except:
        pass
    return np.nan

In [58]:
get_state_from_city('ATLANTA').strip()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('ATLANTA',), **{}).
Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\urllib\request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1298, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1344, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1293, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1052, in _send_output
    self.send(msg)
  File "C:\Users\Lenovo\AppData\Local\Programs\Pytho

'Georgia'

In [59]:
# US states full name to abbreviation mapping
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    'District of Columbia': 'DC'
}


In [60]:
# Function to map to the abbreation form
def extract_state_abbr_from_address(full_address):
    return us_state_abbrev.get(full_address, np.nan)

In [61]:
extract_state_abbr_from_address(get_state_from_city('ATLANTA').strip())

'GA'

In [62]:
# Function to apply to null state values
def fill_state(row):
    if pd.isnull(row['State']):
        state_name = get_state_from_city(row['City']).strip()
        if state_name != "Unknown":
            return extract_state_abbr_from_address(state_name)
    return row['State']

# Apply the function only to rows where 'state' is null
df['State'] = df.apply(fill_state, axis=1)

In [63]:
df['State'].isnull().sum()

np.int64(0)

In [64]:
df[df['City']=='ATLANTA']

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
6,ATLANTA US PEN,ATLANTA,GA,17.341317,3.610778,3.053892,2.724551,NaT


In [65]:
df[df['City']=='ENCINAL']

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
71,LA SALLE COUNTY REGIONAL DETENTION CENTER,ENCINAL,TX,8.91018,0.706587,0.0,0.0719,2025-02-07


#### Checking City Columns

In [68]:
# Replace 'N/A', 'NA', and blanks with proper NaN
df['City'] = df['City'].replace(
    ['N/A', 'NA', ' ', ''], pd.NA
)

df.City.isnull().sum()

np.int64(1)

In [66]:
df.City.isnull().sum()

np.int64(0)

In [69]:
# Now remove rows where City is null
df = df.dropna(subset=['City'])

In [70]:
df.count()

Name                        131
City                        131
State                       131
Level A                     131
Level B                     131
Level C                     131
Level D                     131
Last Inspection End Date    113
dtype: int64

#### Checking the Cleaned data frame

In [71]:
df.describe()

,Level A,Level B,Level C,Level D,Last Inspection End Date
count,131.000000,131.000000,131.000000,131.000000,113
mean,186.810760,40.864469,45.272478,34.404947,2024-09-24 06:09:33.451327488
min,0.000000,0.000000,0.000000,0.000000,2021-04-02 00:00:00
25%,2.125749,1.613772,1.221557,0.685629,2024-08-15 00:00:00
50%,17.017964,8.371257,6.880240,4.940120,2024-10-31 00:00:00
75%,222.790419,39.631737,38.997006,35.452096,2025-01-16 00:00:00
max,1876.461078,561.425150,542.275449,402.059880,2025-03-13 00:00:00
std,320.650168,79.973686,85.286985,65.432270,NaN


In [72]:
df.head()

,Name,City,State,Level A,Level B,Level C,Level D,Last Inspection End Date
0,ADAMS COUNTY DET CENTER,NATCHEZ,MS,1876.461078,266.431138,6.724551,4.257485,2025-01-16
1,ADELANTO ICE PROCESSING CENTER,ADELANTO,CA,6.401198,4.323353,22.604790,32.634731,2024-07-18
2,ALAMANCE COUNTY DETENTION FACILITY,GRAHAM,NC,4.329341,3.461078,6.083832,6.185629,2024-09-19
3,ALEXANDRIA STAGING FACILITY,ALEXANDRIA,LA,137.520958,47.413174,76.005988,52.485030,2024-08-29
4,ALLEGANY COUNTY JAIL,BELMONT,NY,1.221557,0.018000,0.000000,0.000000,NaT


In [73]:
# saving the cleaned data frame
df.to_csv('../cleaned_ice_detention.csv',encoding='utf-8')